In [ ]:
#pip install streamlit scikit-learn pandas numpy sentence-transformers

In [12]:
#pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------- -------------------- 0.8/1.6 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install sentence-transformers scikit-learn

In [43]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import pickle

In [3]:
# Load the transactions sheet in pandas dataframe
# Read only required columns to save memory
dft = pd.read_excel(r'C:\Users\pravi\Documents\Study\HSBC RC Hackathon 2025\Hackathon-2025-dev\Vikash\transactions_sample_1.xlsx', 
                    sheet_name='transactions_sample', 
                    usecols=['originator_company','originator_country','beneficiary_company','beneficiary_country','amount','currency','date'])


In [4]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   originator_company   2000 non-null   object        
 1   originator_country   2000 non-null   object        
 2   beneficiary_company  2000 non-null   object        
 3   beneficiary_country  2000 non-null   object        
 4   amount               2000 non-null   float64       
 5   currency             2000 non-null   object        
 6   date                 2000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 109.5+ KB


In [5]:
dft.head()

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date
0,"Deleon, Lewis and Myers",Saint Helena,Golden-Fry,Solomon Islands,33807.85,USD,2025-05-31
1,Soto Group,Antigua and Barbuda,Brown Group,Syria,29141.30,USD,2025-07-04
2,Harrison PLC,Cambodia,"Best, Berg and Charles",Kyrgyz Republic,2749.30,USD,2025-07-01
3,Baxter LLC,Costa Rica,Baxter LLC,Anguilla,23965.96,USD,2025-04-28
4,"Herrera, Davis and Alexander",Sri Lanka,"Herrera, Davis and Alexander",Azerbaijan,35091.88,USD,2025-03-17


In [9]:
# Load references sheet into dataframe
dfr = pd.read_csv(r'C:\Users\pravi\Documents\Study\HSBC RC Hackathon 2025\Hackathon-2025-dev\Vikash\registry_sample.csv')

In [10]:
dfr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_id    500 non-null    object
 1   company_name  500 non-null    object
 2   country       500 non-null    object
 3   SIC_code      500 non-null    int64 
 4   SIC_desc      500 non-null    object
 5   NACE_code     500 non-null    object
dtypes: int64(1), object(5)
memory usage: 23.6+ KB


In [11]:
dfr.head()

,company_id,company_name,country,SIC_code,SIC_desc,NACE_code
0,4fb8575a-1c6d-43ec-a738-7399e2b7d09c,"Ortiz, Williams and Smith",Tokelau,3020,Manufacture of railway locomotives,C30
1,06a37225-4205-4c3c-bbe8-d6dc52507668,"Brown, Cooke and Webster",Syrian Arab Republic,8622,Specialist medical practice activities,Q86
2,77e1ec10-4f7e-4aef-865a-2212a8d271ea,Mullins Inc,Palau,8622,Specialist medical practice activities,Q86
3,7448a02e-98f6-45c8-ab9d-07f0e0b6aba1,"Pham, Watson and Jackson",Armenia,3511,Electric power generation,D35
4,6f1f367d-25a8-46e6-809b-a188a56ef037,Burns-Russell,Poland,3511,Electric power generation,D35


In [19]:
# Are there any duplicates ?
dups = dfr['company_name'].duplicated(keep=False)
print('Number of duplicate rows = %d' % (dups.sum()))
dfr[dups]

Number of duplicate rows = 12


,company_id,company_name,country,SIC_code,SIC_desc,NACE_code
44,db943c2a-33fd-4cba-9674-923ed67a3aea,Jordan Inc,Dominican Republic,6201,Computer programming activities,J62
46,e8862713-3d89-4944-a879-cc80be98bdfe,Brown and Sons,Fiji,4711,Retail sale in non-specialized stores,G47
62,d4eebc27-fdfc-4ef3-aa1d-b22d4bc39feb,Harris Ltd,Djibouti,4920,Freight transport by road,H49
88,ed63cacc-d82f-4519-b5f2-bfc358405036,Rodriguez Ltd,Philippines,6419,Other monetary intermediation,K64
98,8b39f65a-bb29-4068-83c6-64847a6cadcd,Jordan Inc,Congo,6419,Other monetary intermediation,K64
132,e85eac14-221c-4c38-8724-7c91b864c557,Brown and Sons,Japan,111,Crop and animal production,A01
211,5bbe4135-9cf9-4d18-a442-c707554dbf2b,Taylor Inc,Andorra,6201,Computer programming activities,J62
270,839d5a3e-dfd3-4700-936a-534468f6b0d0,Rodriguez Ltd,Finland,6419,Other monetary intermediation,K64
316,2cef45ed-5638-43d2-a7fc-d764d3387959,Taylor Inc,Micronesia,3020,Manufacture of railway locomotives,C30
357,7913f47b-35eb-42f9-a985-9551901c6a10,Brown and Sons,Barbados,6201,Computer programming activities,J62


In [20]:
# Creating backup copies
dft_bkp_1 = dft.copy()
dfr_bkp_1 = dfr.copy()

In [59]:
# Revert back
#dft = dft_bkp_1.copy()
#dfr = dfr_bkp_1.copy()

In [21]:
# Extract SIC code for an originator and beneficiary based on fuzzy logic on name and exact logic on country
# https://medium.com/@kasperjuunge/rapidfuzz-explained-c26e93b6012d

def fuzzy_match_id(name, country, df_ref, scorer=fuzz.WRatio, score_cutoff=80):
    if pd.isna(name) or pd.isna(country):
        return pd.NA
    
    # Filter df_ref by exact country match
    candidates = dfr[dfr['country'] == country]
    if candidates.empty:
        return pd.NA
    
    # Extract list of candidate names
    candidate_names = candidates['company_name'].tolist()
    
    # Fuzzy match the name against candidate names
    match = process.extractOne(name, candidate_names, scorer=scorer, score_cutoff=score_cutoff)
    if match:
        best_name, score, _ = match
        # Return id for the best matched name with the same country
        matched_id = candidates.loc[candidates['company_name'] == best_name, 'SIC_code'].iloc[0]
        return matched_id
    else:
        return pd.NA

# Apply fuzzy match lookup for originator
dft['originator_id'] = dft.apply(
    lambda row: fuzzy_match_id(row['originator_company'], row['originator_country'], dfr), axis=1)

# Apply fuzzy match lookup for beneficiary
dft['beneficiary_id'] = dft.apply(
    lambda row: fuzzy_match_id(row['beneficiary_company'], row['beneficiary_country'], dfr), axis=1)

# Result df1 has new columns originator_id and beneficiary_id
print(dft.head())



             originator_company   originator_country  \
0       Deleon, Lewis and Myers         Saint Helena   
1                    Soto Group  Antigua and Barbuda   
2                  Harrison PLC             Cambodia   
3                    Baxter LLC           Costa Rica   
4  Herrera, Davis and Alexander            Sri Lanka   

            beneficiary_company beneficiary_country    amount currency  \
0                    Golden-Fry     Solomon Islands  33807.85      USD   
1                   Brown Group               Syria  29141.30      USD   
2        Best, Berg and Charles     Kyrgyz Republic   2749.30      USD   
3                    Baxter LLC            Anguilla  23965.96      USD   
4  Herrera, Davis and Alexander          Azerbaijan  35091.88      USD   

        date  originator_id beneficiary_id  
0 2025-05-31           3511           3511  
1 2025-07-04           6201           <NA>  
2 2025-07-01           3020           4920  
3 2025-04-28           3020           

In [64]:
# Extract SIC code for an originator and beneficiary based on fuzzy logic on name and exact logic on country
# https://medium.com/@kasperjuunge/rapidfuzz-explained-c26e93b6012d

def fuzzy_match_id(name, country, df_ref, scorer=fuzz.WRatio, score_cutoff=80):
    if pd.isna(name) or pd.isna(country):
        return pd.NA, pd.NA
    
    # Filter df_ref by exact country match
    candidates = dfr[dfr['country'] == country]
    if candidates.empty:
        return pd.NA, pd.NA
    
    # Extract list of candidate names
    candidate_names = candidates['company_name'].tolist()
    
    # Fuzzy match the name against candidate names
    match = process.extractOne(name, candidate_names, scorer=scorer, score_cutoff=score_cutoff)
    if match:
        best_name, score, _ = match
        matched_row = candidates[candidates['company_name'] == best_name].iloc[0]
        matched_id = matched_row['SIC_code']
        description = matched_row.get('SIC_desc', pd.NA)  # Use pd.NA if Description is missing
        return matched_id, description
    else:
        return pd.NA, pd.NA

# Apply fuzzy match lookup for originator
dft[['originator_id','originator_desc']] = dft.apply(
    lambda row: pd.Series(fuzzy_match_id(row['originator_company'], row['originator_country'], dfr)), axis=1)

# Apply fuzzy match lookup for beneficiary
dft[['beneficiary_id','beneficiary_desc']] = dft.apply(
    lambda row: pd.Series(fuzzy_match_id(row['beneficiary_company'], row['beneficiary_country'], dfr)), axis=1)


In [65]:
dft.head(10)

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date,originator_id,originator_desc,beneficiary_id,beneficiary_desc
0,"Deleon, Lewis and Myers",Saint Helena,Golden-Fry,Solomon Islands,33807.85,USD,2025-05-31,3511,Electric power generation,3511,Electric power generation
1,Soto Group,Antigua and Barbuda,Brown Group,Syria,29141.30,USD,2025-07-04,6201,Computer programming activities,<NA>,<NA>
2,Harrison PLC,Cambodia,"Best, Berg and Charles",Kyrgyz Republic,2749.30,USD,2025-07-01,3020,Manufacture of railway locomotives,4920,Freight transport by road
3,Baxter LLC,Costa Rica,Baxter LLC,Anguilla,23965.96,USD,2025-04-28,3020,Manufacture of railway locomotives,<NA>,<NA>
4,"Herrera, Davis and Alexander",Sri Lanka,"Herrera, Davis and Alexander",Azerbaijan,35091.88,USD,2025-03-17,4711,Retail sale in non-specialized stores,<NA>,<NA>
5,"Torres, Romero and Bell",Isle of Man,Bowman Group,Jordan,5626.92,USD,2025-07-11,4711,Retail sale in non-specialized stores,3511,Electric power generation
6,Anderson Group,Holy See (Vatican City State),Patterson-Guerrero,Antigua and Barbuda,28288.50,USD,2025-02-06,111,Crop and animal production,8622,Specialist medical practice activities
7,"Hooper, Rodriguez and White",Bermuda,"Hooper, Rodriguez and White",Gabon,24013.97,USD,2025-04-03,4711,Retail sale in non-specialized stores,<NA>,<NA>
8,Manning-Rojas,Monaco,Manning-Rojas,Ethiopia,18892.53,USD,2025-06-27,6419,Other monetary intermediation,<NA>,<NA>
9,Dunn PLC,Namibia,Dunn PLC,Poland,18819.74,USD,2025-05-01,6201,Computer programming activities,<NA>,<NA>


The NA cases in the above df are cases where the company cannot be found in the country.</br>
So we can have different scenarios for this. If country is high risk, raise an alert or check if the company exists in another country and check company description etc.

In [66]:
# Checking NA records 
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   originator_company   2000 non-null   object        
 1   originator_country   2000 non-null   object        
 2   beneficiary_company  2000 non-null   object        
 3   beneficiary_country  2000 non-null   object        
 4   amount               2000 non-null   float64       
 5   currency             2000 non-null   object        
 6   date                 2000 non-null   datetime64[ns]
 7   originator_id        2000 non-null   int64         
 8   originator_desc      2000 non-null   object        
 9   beneficiary_id       1723 non-null   object        
 10  beneficiary_desc     1723 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 172.0+ KB


In [67]:
rows_with_na = dft[dft['originator_id'].isna() | dft['beneficiary_id'].isna()]
rows_with_na

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date,originator_id,originator_desc,beneficiary_id,beneficiary_desc
1,Soto Group,Antigua and Barbuda,Brown Group,Syria,29141.30,USD,2025-07-04,6201,Computer programming activities,<NA>,<NA>
3,Baxter LLC,Costa Rica,Baxter LLC,Anguilla,23965.96,USD,2025-04-28,3020,Manufacture of railway locomotives,<NA>,<NA>
4,"Herrera, Davis and Alexander",Sri Lanka,"Herrera, Davis and Alexander",Azerbaijan,35091.88,USD,2025-03-17,4711,Retail sale in non-specialized stores,<NA>,<NA>
7,"Hooper, Rodriguez and White",Bermuda,"Hooper, Rodriguez and White",Gabon,24013.97,USD,2025-04-03,4711,Retail sale in non-specialized stores,<NA>,<NA>
8,Manning-Rojas,Monaco,Manning-Rojas,Ethiopia,18892.53,USD,2025-06-27,6419,Other monetary intermediation,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...
1979,Allen Group,Rwanda,Merritt and Sons,North Korea,6903.55,USD,2025-02-12,3020,Manufacture of railway locomotives,<NA>,<NA>
1982,"Todd, Ferguson and Cole",Malaysia,Nguyen-James,Iran,21296.69,USD,2025-03-20,3020,Manufacture of railway locomotives,<NA>,<NA>
1988,Rogers Group,Fiji,"Brown, Cooke and Webster",North Korea,49203.08,USD,2025-07-13,3511,Electric power generation,<NA>,<NA>
1990,"Herrera, Davis and Alexander",Sri Lanka,"Lee, Reyes and Taylor",Iran,29193.23,USD,2025-05-23,4711,Retail sale in non-specialized stores,<NA>,<NA>


In [68]:
dft[dft['originator_id'].isna()].count().iloc[0]

0

In [69]:
dft[dft['beneficiary_id'].isna()].count().iloc[0]

277

In [70]:
# Shape before
dft.shape

(2000, 11)

In [71]:
dft_ids = dft.dropna()
print("New df:", dft_ids.shape, "Existing df:", dft.shape)

New df: (1723, 11) Existing df: (2000, 11)


In [72]:
dft_ids.head()

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date,originator_id,originator_desc,beneficiary_id,beneficiary_desc
0,"Deleon, Lewis and Myers",Saint Helena,Golden-Fry,Solomon Islands,33807.85,USD,2025-05-31,3511,Electric power generation,3511,Electric power generation
2,Harrison PLC,Cambodia,"Best, Berg and Charles",Kyrgyz Republic,2749.30,USD,2025-07-01,3020,Manufacture of railway locomotives,4920,Freight transport by road
5,"Torres, Romero and Bell",Isle of Man,Bowman Group,Jordan,5626.92,USD,2025-07-11,4711,Retail sale in non-specialized stores,3511,Electric power generation
6,Anderson Group,Holy See (Vatican City State),Patterson-Guerrero,Antigua and Barbuda,28288.50,USD,2025-02-06,111,Crop and animal production,8622,Specialist medical practice activities
10,Taylor Ltd,Hong Kong,Greene-Beard,Guinea-Bissau,15683.39,USD,2025-07-24,4920,Freight transport by road,3020,Manufacture of railway locomotives


Now there are multiple ways to deal with determinig if transaction makes sense or not.</br>
<li>SIC codes list contains Division, Major Group, Industry Group. Simple lookups can be used for same groups</li>
<li>Clustering based on any of these groups and determine if the SIC code belongs to same cluster</li>
<li>Use description and natural language processing and determine cluster based on given description</li>

Below section is a Python workflow that uses NLP (sentence embeddings) to cluster SIC code descriptions and a function to determine if any two provided descriptions belong to the same cluster.</br>
The code leverages the sentence-transformers library (for embeddings) and KMeans (for clustering).

In [44]:
# 1. Load Data
df_sic = pd.read_csv(r'C:\Users\pravi\Documents\Study\HSBC RC Hackathon 2025\Hackathon-2025-dev\sic4-list-master_saintsjd\sic-codes.csv')
df_sic.head()

,Division,Major Group,Industry Group,SIC,Description
0,A,1,11,111,Wheat
1,A,1,11,112,Rice
2,A,1,11,115,Corn
3,A,1,11,116,Soybeans
4,A,1,11,119,"Cash Grains, Not Elsewhere Classified"


In [45]:
df_sic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Division        1005 non-null   object
 1   Major Group     1005 non-null   int64 
 2   Industry Group  1005 non-null   int64 
 3   SIC             1005 non-null   int64 
 4   Description     1005 non-null   object
dtypes: int64(3), object(2)
memory usage: 39.4+ KB


In [49]:
# Collect all descriptions
descriptions = df_sic['Description'].astype(str).tolist()
descriptions[0:10]

['Wheat',
 'Rice',
 'Corn',
 'Soybeans',
 'Cash Grains, Not Elsewhere Classified',
 'Cotton',
 'Tobacco',
 'Sugarcane and Sugar Beets',
 'Irish Potatoes',
 'Field Crops, Except Cash Grains, Not Elsewhere Classified']

In [51]:
# 2. Generate Embeddings
# Download a small but accurate model for clustering
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(descriptions)


In [53]:
# 3. Clustering
# You can set n_clusters to the count of unique Divisions, or tune with other methods
n_clusters = len(df_sic['Division'].unique())
kmeans = KMeans(n_clusters=n_clusters, random_state=21)
clusters = kmeans.fit_predict(embeddings)
df_sic['nlp_cluster'] = clusters


C:\Users\pravi\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [55]:
df_sic.head(10)

,Division,Major Group,Industry Group,SIC,Description,nlp_cluster
0,A,1,11,111,Wheat,7
1,A,1,11,112,Rice,7
2,A,1,11,115,Corn,7
3,A,1,11,116,Soybeans,7
4,A,1,11,119,"Cash Grains, Not Elsewhere Classified",0
5,A,1,13,131,Cotton,1
6,A,1,13,132,Tobacco,7
7,A,1,13,133,Sugarcane and Sugar Beets,7
8,A,1,13,134,Irish Potatoes,7
9,A,1,13,139,"Field Crops, Except Cash Grains, Not Elsewhere...",0


In [56]:
df_sic['nlp_cluster'].unique()

array([7, 0, 1, 9, 5, 3, 4, 8, 2, 6])

In [57]:
# 4. Save Model Assets
with open('sic_nlp_cluster.pkl', 'wb') as f:
    pickle.dump({'model': model, 'kmeans': kmeans, 'cluster_map': clusters}, f)

print("NLP cluster model saved as sic_nlp_cluster.pkl")


NLP cluster model saved as sic_nlp_cluster.pkl


In [58]:
# 5. Function to Check if Two Descriptions Belong to Same Cluster
def same_cluster_nlp(desc1, desc2):
    with open('sic_nlp_cluster.pkl', 'rb') as f:
        assets = pickle.load(f)
    model = assets['model']
    kmeans = assets['kmeans']
    # Embed both descriptions
    X = model.encode([desc1, desc2])
    # Predict cluster ID for both
    cl1, cl2 = kmeans.predict(X)
    return cl1 == cl2, cl1, cl2


In [ ]:
# Example Usage:
desc_a = "Crude Petroleum and Natural Gas"
desc_b = "Natural Gas Liquids"
same, c1, c2 = same_cluster_nlp(desc_a, desc_b)
print(f"Description 1 cluster: {c1}")
print(f"Description 2 cluster: {c2}")
print("Same cluster?", same)

desc_c = "Hotels and Motels"
same, c1, c2 = same_cluster_nlp(desc_a, desc_c)
print(f"Description 3 cluster: {c2}")
print("Same cluster with first?", same)

In [75]:
# Example Usage 1:
desc_a = dft_ids['originator_desc'].iloc[0]
desc_b = dft_ids['beneficiary_desc'].iloc[0]
print(desc_a," ",desc_b)
same, c1, c2 = same_cluster_nlp(desc_a, desc_b)
print(f"Description 1 cluster: {c1}")
print(f"Description 2 cluster: {c2}")
print("Same cluster?", same)

desc_c = dft_ids['beneficiary_desc'].iloc[1]
print(desc_a," ",desc_c)
same, c1, c2 = same_cluster_nlp(desc_a, desc_c)
print(f"Description 3 cluster: {c2}")
print("Same cluster with first?", same)

Electric power generation   Electric power generation
Description 1 cluster: 2
Description 2 cluster: 2
Same cluster? True
Electric power generation   Freight transport by road
Description 3 cluster: 8
Same cluster with first? False
